In [1]:
import pandas as pd
import numpy as np
import os
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pickle
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight

In [2]:
hikari = pd.read_csv("/mnt/c/users/razaa/downloads/ALLFLOWMETER_HIKARI2021.csv")

In [3]:
hikari.drop(columns=["Unnamed: 0.1", "Unnamed: 0"], inplace=True)

In [4]:
hikari

uid        originh  originp          responh  responp  \
0        Cg61Jch3vdz9DBptj  103.255.15.23    13316  128.199.242.104      443   
1        CdRIlqLWdj35Y9vW9  103.255.15.23    13318  128.199.242.104      443   
2        CLzp9Khd0Y09Qkgrg  103.255.15.23    13320  128.199.242.104      443   
3       Cnf1YA4iLB4CSNWB88  103.255.15.23    13322  128.199.242.104      443   
4       C4ZKvv3fpO72EAOsJ6  103.255.15.23    13324  128.199.242.104      443   
...                    ...            ...      ...              ...      ...   
555273  C9b6Aa2csiogu3vVp9  103.255.15.42      138   103.255.15.255      138   
555274  CGDT4r4PAbp3mvaI6k  103.255.15.42      138   103.255.15.255      138   
555275  CJUxTk4Qd0kHliUKR9  103.255.15.42      138   103.255.15.255      138   
555276  CknUJi2R1iYJG3li3k  103.255.15.42      138   103.255.15.255      138   
555277  C82mlb2i3zEzpeTGQk  103.255.15.42      138   103.255.15.255      138   

        flow_duration  fwd_pkts_tot  bwd_pkts_tot  fwd_data_pkts_tot  \
0            2.207588            15            14                  6   
1           15.624266            15            14                  6   
2           12.203357            14            13                  6   
3            9.992448            14            13                  6   
4            7.780611            14            14                  6   
...               ...           ...           ...                ...   
555273       0.000000             1             0                  1   
555274       0.000000             1             0                  1   
555275       0.000000             1             0                  1   
555276       0.000000             1             0                  1   
555277       0.000000             1             0                  1   

        bwd_data_pkts_tot  ...      idle.min      idle.max      idle.tot  \
0                       6  ...  0.000000e+00  0.000000e+00  0.000000e+00   
1                       6  ...  1.534300e+07  1.534300e+07  1.534300e+07   
2                       5  ...  1.196814e+07  1.196814e+07  1.196814e+07   
3                       5  ...  9.759205e+06  9.759205e+06  9.759205e+06   
4                       5  ...  7.545305e+06  7.545305e+06  7.545305e+06   
...                   ...  ...           ...           ...           ...   
555273                  0  ...  0.000000e+00  0.000000e+00  0.000000e+00   
555274                  0  ...  0.000000e+00  0.000000e+00  0.000000e+00   
555275                  0  ...  0.000000e+00  0.000000e+00  0.000000e+00   
555276                  0  ...  0.000000e+00  0.000000e+00  0.000000e+00   
555277                  0  ...  0.000000e+00  0.000000e+00  0.000000e+00   

            idle.avg  idle.std  fwd_init_window_size  bwd_init_window_size  \
0       0.000000e+00       0.0                 29200                 65160   
1       1.534300e+07       0.0                 29200                 65160   
2       1.196814e+07       0.0                 29200                 65160   
3       9.759205e+06       0.0                 29200                 65160   
4       7.545305e+06       0.0                 29200                 65160   
...              ...       ...                   ...                   ...   
555273  0.000000e+00       0.0                     0                     0   
555274  0.000000e+00       0.0                     0                     0   
555275  0.000000e+00       0.0                     0                     0   
555276  0.000000e+00       0.0                     0                     0   
555277  0.000000e+00       0.0                     0                     0   

        fwd_last_window_size     traffic_category  Label  
0                          0       Bruteforce-XML      1  
1                          0       Bruteforce-XML      1  
2                          0       Bruteforce-XML      1  
3                          0       Bruteforce-XML      1  
4                          0       Bruteforce-XML   

In [5]:
PATH_TO_CIC = "/mnt/c/users/razaa/downloads/CIC-DDoS2019"
cic_paths = os.listdir(PATH_TO_CIC)
cic_train_paths = [os.path.join(PATH_TO_CIC, path) for path in cic_paths if "-training" in path]
cic_test_paths = [os.path.join(PATH_TO_CIC, path) for path in cic_paths if "-testing" in path]


train_prefixes = [path.split('/')[-1].split('-')[0] for path in cic_train_paths]
test_prefixes = [path.split('/')[-1].split('-')[0] for path in cic_test_paths]

common_prefixes = list(set(train_prefixes).intersection(test_prefixes))

cic_train_paths = [path for path in cic_train_paths if path.split('/')[-1].split('-')[0] in common_prefixes]
cic_test_paths = [path for path in cic_test_paths if path.split('/')[-1].split('-')[0] in common_prefixes]

cic_train = pd.concat([pd.read_parquet(path) for path in cic_train_paths], ignore_index=True)
cic_test = pd.concat([pd.read_parquet(path) for path in cic_test_paths], ignore_index=True)

In [6]:
cic_train

Protocol  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0             17             49                  2                       0   
1             17              1                  2                       0   
2             17              1                  2                       0   
3             17              1                  2                       0   
4             17              1                  2                       0   
...          ...            ...                ...                     ...   
120060         6             53                  1                       2   
120061         6             66                  1                       2   
120062        17          26288                  2                       2   
120063         6          30679                  1                       1   
120064         6             67                  1                       2   

        Fwd Packets Length Total  Bwd Packets Length Total  \
0                          458.0                       0.0   
1                         2944.0                       0.0   
2                          458.0                       0.0   
3                         2944.0                       0.0   
4                         2944.0                       0.0   
...                          ...                       ...   
120060                       0.0                       0.0   
120061                       0.0                       0.0   
120062                      96.0                      96.0   
120063                       0.0                       0.0   
120064                       0.0                       0.0   

        Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                       229.0                  229.0                   229.0   
1                      1472.0                 1472.0                  1472.0   
2                       229.0                  229.0                   229.0   
3                      1472.0                 1472.0                  1472.0   
4                      1472.0                 1472.0                  1472.0   
...                       ...                    ...                     ...   
120060                    0.0                    0.0                     0.0   
120061                    0.0                    0.0                     0.0   
120062                   48.0                   48.0                    48.0   
120063                    0.0                    0.0                     0.0   
120064                    0.0                    0.0                     0.0   

        Fwd Packet Length Std  ...  Fwd Seg Size Min  Active Mean  Active Std  \
0                         0.0  ...                 8          0.0         0.0   
1                         0.0  ...              1480          0.0         0.0   
2                         0.0  ...                14          0.0         0.0   
3                         0.0  ...                14          0.0         0.0   
4                         0.0  ...                32          0.0         0.0   
...                       ...  ...               ...          ...         ...   
120060                    0.0  ...                32          0.0         0.0   
120061                    0.0  ...                32          0.0         0.0   
120062                    0.0  ...                20          0.0         0.0   
120063                    0.0  ...                32          0.0         0.0   
120064                    0.0  ...                32          0.0         0.0   

        Active Max  Active Min  Idle Mean  Idle Std  Idle Max  Idle Min  \
0              0.0         0.0        0.0       0.0       0.0       0.0   
1              0.0         0.0        0.0       0.0       0.0       0.0   
2              0.0         0.0        0.0       0.0       0.0       0.0   
3              0.0         0.0        0.0       0.0       0.0       0.0   
4              0.

In [7]:
cic_train.columns

Index(['Protocol', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Fwd Packets Length Total',
       'Bwd Packets Length Total', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Fla

In [8]:
hikari.columns

Index(['uid', 'originh', 'originp', 'responh', 'responp', 'flow_duration',
       'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot',
       'bwd_data_pkts_tot', 'fwd_pkts_per_sec', 'bwd_pkts_per_sec',
       'flow_pkts_per_sec', 'down_up_ratio', 'fwd_header_size_tot',
       'fwd_header_size_min', 'fwd_header_size_max', 'bwd_header_size_tot',
       'bwd_header_size_min', 'bwd_header_size_max', 'flow_FIN_flag_count',
       'flow_SYN_flag_count', 'flow_RST_flag_count', 'fwd_PSH_flag_count',
       'bwd_PSH_flag_count', 'flow_ACK_flag_count', 'fwd_URG_flag_count',
       'bwd_URG_flag_count', 'flow_CWR_flag_count', 'flow_ECE_flag_count',
       'fwd_pkts_payload.min', 'fwd_pkts_payload.max', 'fwd_pkts_payload.tot',
       'fwd_pkts_payload.avg', 'fwd_pkts_payload.std', 'bwd_pkts_payload.min',
       'bwd_pkts_payload.max', 'bwd_pkts_payload.tot', 'bwd_pkts_payload.avg',
       'bwd_pkts_payload.std', 'flow_pkts_payload.min',
       'flow_pkts_payload.max', 'flow_pkts_payload.tot',
   

In [9]:
hikari_to_cic_keys = {
    'flow_duration': 'Flow Duration',
    'fwd_pkts_tot': 'Total Fwd Packets',
    'bwd_pkts_tot': 'Total Backward Packets',
    'fwd_data_pkts_tot': 'Fwd Packets Length Total',
    'bwd_data_pkts_tot': 'Bwd Packets Length Total',
    'fwd_pkts_per_sec': 'Fwd Packets/s',
    'bwd_pkts_per_sec': 'Bwd Packets/s',
    'down_up_ratio': 'Down/Up Ratio',
    'fwd_header_size_tot': 'Fwd Header Length',
    'bwd_header_size_tot': 'Bwd Header Length',
    'flow_pkts_per_sec': 'Flow Packets/s',
    'flow_iat.avg': 'Flow IAT Mean',
    'flow_iat.std': 'Flow IAT Std',
    'flow_iat.max': 'Flow IAT Max',
    'flow_iat.min': 'Flow IAT Min',
    'fwd_iat.tot': 'Fwd IAT Total', 
    'fwd_iat.max': 'Fwd IAT Max',
    'fwd_iat.avg': 'Fwd IAT Mean',
    'fwd_iat.std': 'Fwd IAT Std',
    'fwd_iat.min': 'Fwd IAT Min',
    'bwd_iat.tot': 'Bwd IAT Total',
    'bwd_iat.avg': 'Bwd IAT Mean',
    'bwd_iat.max': 'Bwd IAT Max',
    'bwd_iat.min': 'Bwd IAT Min',
    'bwd_iat.std': 'Bwd IAT Std',
    'flow_SYN_flag_count': 'SYN Flag Count',
    'flow_RST_flag_count': 'RST Flag Count',
    'fwd_PSH_flag_count': 'Fwd PSH Flags',
    'bwd_PSH_flag_count': 'Bwd PSH Flags',
    'flow_ACK_flag_count': 'ACK Flag Count',
    'fwd_URG_flag_count': 'Fwd URG Flags',
    'bwd_URG_flag_count': 'Bwd URG Flags',
    'flow_ECE_flag_count': 'ECE Flag Count',
    'fwd_bulk_rate': 'Fwd Avg Bulk Rate',
    'bwd_bulk_rate': 'Bwd Avg Bulk Rate',
    'fwd_bulk_packets': 'Fwd Avg Packets/Bulk',
    'bwd_bulk_packets': 'Bwd Avg Packets/Bulk',
    'fwd_bulk_bytes': 'Fwd Avg Bytes/Bulk',
    'bwd_bulk_bytes': 'Bwd Avg Bytes/Bulk',
    'active.avg': 'Active Mean',
    'active.min': 'Active Min',
    'active.max': 'Active Max',
    'active.std': 'Active Std',
    'idle.avg': 'Idle Mean',
    'idle.min': 'Idle Min',
    'idle.max': 'Idle Max',
    'idle.std': 'Idle Std',
    'fwd_init_window_size': 'Init Fwd Win Bytes',
    'bwd_init_window_size': 'Init Bwd Win Bytes',
    'fwd_subflow_pkts': 'Subflow Fwd Packets',
    'bwd_subflow_pkts': 'Subflow Bwd Packets',
    'fwd_subflow_bytes': 'Subflow Fwd Bytes',
    'bwd_subflow_bytes': 'Subflow Bwd Bytes',
    'Label': 'Label'
}

In [10]:
hikari_filtered = hikari.loc[:, hikari.columns.intersection(hikari_to_cic_keys.keys())]

In [11]:
hikari_cic = hikari_filtered.rename(columns=hikari_to_cic_keys)

In [12]:
cic_train = cic_train.loc[:, cic_train.columns.intersection(hikari_to_cic_keys.values())]

In [13]:
cic_test = cic_test.loc[:, cic_test.columns.intersection(hikari_to_cic_keys.values())]

In [14]:
hikari_train = hikari_cic.sample(frac=0.75, random_state=42)
hikari_test = hikari_cic.drop(hikari_train.index)

In [15]:
label_conversion_dict = cic_train.Label.unique().tolist() + cic_test.Label.unique().tolist()

In [16]:
label_conversion_dict = list(set(label_conversion_dict))
label_conversion_dict = {
    key: 0 if key == "Benign" else 1 for key in label_conversion_dict
}

In [17]:
label_conversion_dict

{'LDAP': 1,
 'DrDoS_UDP': 1,
 'UDP-lag': 1,
 'DrDoS_NetBIOS': 1,
 'NetBIOS': 1,
 'UDP': 1,
 'MSSQL': 1,
 'UDPLag': 1,
 'DrDoS_LDAP': 1,
 'Syn': 1,
 'WebDDoS': 1,
 'DrDoS_MSSQL': 1,
 'Benign': 0}

In [18]:
cic_train["Label"] = cic_train["Label"].replace(label_conversion_dict)

In [19]:
cic_test["Label"] = cic_test["Label"].replace(label_conversion_dict)

In [20]:
combined_train = pd.concat([cic_train, hikari_train])

In [21]:
combined_test = pd.concat([cic_test, hikari_test])

In [22]:
combined_train.to_csv("cic_ddos_and_hikari_train.csv")
combined_test.to_csv("cic_ddos_and_hikari_test.csv")

In [24]:
combined_test

Flow Duration  Total Fwd Packets  Total Backward Packets  \
0            1.000000                  2                       0   
1            1.000000                  2                       0   
2            1.000000                  2                       0   
3            2.000000                  2                       0   
4            1.000000                  2                       0   
...               ...                ...                     ...   
555261       1.528769                  3                       0   
555262       1.528655                  3                       0   
555268       0.000000                  1                       0   
555273       0.000000                  1                       0   
555275       0.000000                  1                       0   

        Fwd Packets Length Total  Bwd Packets Length Total  Flow Packets/s  \
0                         2944.0                       0.0    2.000000e+06   
1                         2944.0                       0.0    2.000000e+06   
2                         2944.0                       0.0    2.000000e+06   
3                         2944.0                       0.0    1.000000e+06   
4                         2944.0                       0.0    2.000000e+06   
...                          ...                       ...             ...   
555261                       3.0                       0.0    1.962363e+00   
555262                       3.0                       0.0    1.962509e+00   
555268                       1.0                       0.0    0.000000e+00   
555273                       1.0                       0.0    0.000000e+00   
555275                       1.0                       0.0    0.000000e+00   

        Flow IAT Mean  Flow IAT Std   Flow IAT Max   Flow IAT Min  ...  \
0            1.000000      0.000000       1.000000       1.000000  ...   
1            1.000000      0.000000       1.000000       1.000000  ...   
2            1.000000      0.000000       1.000000       1.000000  ...   
3            2.000000      0.000000       2.000000       2.000000  ...   
4            1.000000      0.000000       1.000000       1.000000  ...   
...               ...           ...            ...            ...  ...   
555261  764384.508133     16.184390  764395.952225  764373.064041  ...   
555262  764327.526093     72.829754  764379.024506  764276.027679  ...   
555268       0.000000      0.000000       0.000000       0.000000  ...   
555273       0.000000      0.000000       0.000000       0.000000  ...   
555275       0.000000      0.000000       0.000000       0.000000  ...   

        Init Bwd Win Bytes   Active Mean  Active Std    Active Max  \
0                       -1  0.000000e+00         0.0  0.000000e+00   
1                       -1  0.000000e+00         0.0  0.000000e+00   
2                       -1  0.000000e+00         0.0  0.000000e+00   
3                       -1  0.000000e+00         0.0  0.000000e+00   
4                       -1  0.000000e+00         0.0  0.000000e+00   
...                    ...           ...         ...           ...   
555261                   0  1.528769e+06         0.0  1.528769e+06   
555262                   0  1.528655e+06         0.0  1.528655e+06   
555268                   0  0.000000e+00         0.0  0.000000e+00   
555273                   0  0.000000e+00         0.0  0.000000e+00   
555275                   0  0.000000e+00         0.0  0.000000e+00   

          Active Min  Idle Mean  Idle Std  Idle Max  Idle Min  Label  
0       0.000000e+00        0.0       0.0       0.0       0.0      1  
1       0.000000e+00        0.0       0.0       0.0       0.0      1  
2       0.000000e+00        0.0       0.0       0.0       0.0      1  
3       0.000000e+00        0.0       0.0       0.0       0.0      1  
4       0.000000e+00        0.0       0.0       0.0       0.0      1  
...              ...        ...       ...       ...       ...    ...  
555261  1.528769e+06       

In [60]:
combined_train

Flow Duration  Total Fwd Packets  Total Backward Packets  \
0           49.000000                  2                       0   
1            1.000000                  2                       0   
2            1.000000                  2                       0   
3            1.000000                  2                       0   
4            1.000000                  2                       0   
...               ...                ...                     ...   
532281       0.335101                 10                      13   
499196       0.343518                 10                      14   
192219       1.219601                  6                       4   
415757       0.281207                 11                      14   
259771       0.024407                  2                       2   

        Fwd Packets Length Total  Bwd Packets Length Total  Flow Packets/s  \
0                          458.0                       0.0    4.081633e+04   
1                         2944.0                       0.0    2.000000e+06   
2                          458.0                       0.0    2.000000e+06   
3                         2944.0                       0.0    2.000000e+06   
4                         2944.0                       0.0    2.000000e+06   
...                          ...                       ...             ...   
532281                       3.0                       8.0    6.863600e+01   
499196                       3.0                       8.0    6.986533e+01   
192219                       1.0                       1.0    8.199403e+00   
415757                       3.0                       9.0    8.890253e+01   
259771                       2.0                       2.0    1.638864e+02   

        Flow IAT Mean   Flow IAT Std  Flow IAT Max  Flow IAT Min  ...  \
0           49.000000       0.000000  4.900000e+01     49.000000  ...   
1            1.000000       0.000000  1.000000e+00      1.000000  ...   
2            1.000000       0.000000  1.000000e+00      1.000000  ...   
3            1.000000       0.000000  1.000000e+00      1.000000  ...   
4            1.000000       0.000000  1.000000e+00      1.000000  ...   
...               ...            ...           ...           ...  ...   
532281   15231.869437   38170.834127  1.750062e+05      2.861023  ...   
499196   14935.566031   41590.744701  1.979821e+05      4.053116  ...   
192219  135515.001085  394252.047954  1.186762e+06     13.828278  ...   
415757   11716.951927   28458.006807  1.339560e+05      1.907349  ...   
259771    8135.716120   14061.947676  2.437305e+04     15.020370  ...   

        Init Bwd Win Bytes   Active Mean  Active Std    Active Max  \
0                       -1  0.000000e+00         0.0  0.000000e+00   
1                       -1  0.000000e+00         0.0  0.000000e+00   
2                       -1  0.000000e+00         0.0  0.000000e+00   
3                       -1  0.000000e+00         0.0  0.000000e+00   
4                       -1  0.000000e+00         0.0  0.000000e+00   
...                    ...           ...         ...           ...   
532281               65160  3.351011e+05         0.0  3.351011e+05   
499196               65160  3.435180e+05         0.0  3.435180e+05   
192219               65535  1.219635e+06         0.0  1.219635e+06   
415757               65160  2.812068e+05         0.0  2.812068e+05   
259771                   0  2.440715e+04         0.0  2.440715e+04   

          Active Min  Idle Mean  Idle Std  Idle Max  Idle Min  Label  
0       0.000000e+00        0.0       0.0       0.0       0.0      1  
1       0.000000e+00        0.0       0.0       0.0       0.0      1  
2       0.000000e+00        0.0       0.0       0.0       0.0      1  
3       0.000000e+00        0.0       0.0       0.0       0.0      1  
4       0.000000e+00        0.0       0.0       0.0       0.0      1  
...              ...        ...       ...       ...       ...    ...  
532281  3.351011e+05        0.0       0

In [61]:
combined_train['Flow Duration'].mean()

4593430.096343373

In [62]:
combined_test['Idle Std']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
555261    0.0
555262    0.0
555268    0.0
555273    0.0
555275    0.0
Name: Idle Std, Length: 177793, dtype: float64

In [63]:
rename = {
 'Bwd Bulk Rate Avg': 'Bwd Avg Bulk Rate',
 'Bwd Bytes/Bulk Avg': 'Bwd Avg Bytes/Bulk',
 'Bwd Packet/Bulk Avg': 'Bwd Avg Packets/Bulk',
 'Total Length of Bwd Packet': 'Bwd Packets Length Total',
 'Fwd Bulk Rate Avg': 'Fwd Avg Bulk Rate',
 'Fwd Bytes/Bulk Avg': 'Fwd Avg Bytes/Bulk',
 'Fwd Packet/Bulk Avg': 'Fwd Avg Packets/Bulk',
 'Total Length of Fwd Packet': 'Fwd Packets Length Total',
 'Bwd Init Win Bytes': 'Init Bwd Win Bytes',
 'FWD Init Win Bytes': 'Init Fwd Win Bytes',
 'Total Bwd packets': 'Total Backward Packets',
 'Total Fwd Packet': 'Total Fwd Packets'
}

In [64]:
combined_test = pd.read_csv("/mnt/c/users/razaa/downloads/ddos-examples/amp.UDP.UBNT.src10001.pcapng_Flow.csv")

In [65]:
combined_test = combined_test.rename(columns=rename)

In [66]:
combined_test.columns

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Fwd Packets Length Total',
       'Bwd Packets Length Total', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'P

In [67]:
combined_test = combined_test.loc[:, combined_test.columns.intersection(hikari_to_cic_keys.values())]

In [68]:
combined_test

Flow Duration  Total Fwd Packets  Total Backward Packets  \
0            4155006                  6                       0   
1            2736956                  3                       0   
2            3055976                  6                       0   
3            3963174                  8                       0   
4            3967971                  5                       0   
...              ...                ...                     ...   
11466        1600649                  2                       0   
11467        1702926                  2                       0   
11468          19441                  2                       0   
11469        3793872                  7                       0   
11470        3472819                  3                       0   

       Fwd Packets Length Total  Bwd Packets Length Total  Flow Packets/s  \
0                        1332.0                       0.0        1.444041   
1                         462.0                       0.0        1.096108   
2                         924.0                       0.0        1.963366   
3                        1248.0                       0.0        2.018584   
4                         770.0                       0.0        1.260090   
...                         ...                       ...             ...   
11466                     302.0                       0.0        1.249493   
11467                     348.0                       0.0        1.174449   
11468                     342.0                       0.0      102.875366   
11469                    1092.0                       0.0        1.845081   
11470                     528.0                       0.0        0.863852   

       Flow IAT Mean  Flow IAT Std  Flow IAT Max  Flow IAT Min  ...  \
0       8.310012e+05  7.150903e+05     1756286.0       20328.0  ...   
1       1.368478e+06  1.753121e+06     2608122.0      128834.0  ...   
2       6.111952e+05  8.779418e+05     2167663.0      104515.0  ...   
3       5.661677e+05  5.147874e+05     1624357.0       11845.0  ...   
4       9.919928e+05  1.274903e+06     2860094.0       25727.0  ...   
...              ...           ...           ...           ...  ...   
11466   1.600649e+06  0.000000e+00     1600649.0     1600649.0  ...   
11467   1.702926e+06  0.000000e+00     1702926.0     1702926.0  ...   
11468   1.944100e+04  0.000000e+00       19441.0       19441.0  ...   
11469   6.323120e+05  7.286191e+05     2008603.0       19098.0  ...   
11470   1.736410e+06  1.495010e+06     2793541.0      679278.0  ...   

       Init Bwd Win Bytes  Active Mean  Active Std  Active Max  Active Min  \
0                       0            0           0           0           0   
1                       0            0           0           0           0   
2                       0            0           0           0           0   
3                       0            0           0           0           0   
4                       0            0           0           0           0   
...                   ...          ...         ...         ...         ...   
11466                   0            0           0           0           0   
11467                   0            0           0           0           0   
11468                   0            0           0           0           0   
11469                   0            0           0           0           0   
11470                   0            0           0           0           0   

       Idle Mean  Idle Std  Idle Max  Idle Min            Label  
0              0         0         0         0  NeedManualLabel  
1              0         0         0         0  NeedManualLabel  
2              0         0         0         0  NeedManualLabel  
3              0         0         0         0  NeedManualLabel  
4              0         0         0         0  NeedManualLabel  
...          ...       ...       ...       ...              ...  
11466          0         

In [69]:
combined_train['Flow Duration']

0         49.000000
1          1.000000
2          1.000000
3          1.000000
4          1.000000
            ...    
532281     0.335101
499196     0.343518
192219     1.219601
415757     0.281207
259771     0.024407
Name: Flow Duration, Length: 536523, dtype: float64

In [70]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),  
            nn.ReLU(),
            nn.Linear(128, latent_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
            # no activation (assuming using MSELoss for reconstruction)
        )
    def forward(self, x):
        latent = self.encoder(x)
        recon = self.decoder(latent)
        return recon, latent

In [71]:
labels_train = combined_train["Label"]
labels_test = combined_test["Label"]

In [72]:
combined_train.drop(columns=["Label"], inplace=True)
combined_test.drop(columns=["Label"], inplace=True)

In [73]:
combined_test

Flow Duration  Total Fwd Packets  Total Backward Packets  \
0            4155006                  6                       0   
1            2736956                  3                       0   
2            3055976                  6                       0   
3            3963174                  8                       0   
4            3967971                  5                       0   
...              ...                ...                     ...   
11466        1600649                  2                       0   
11467        1702926                  2                       0   
11468          19441                  2                       0   
11469        3793872                  7                       0   
11470        3472819                  3                       0   

       Fwd Packets Length Total  Bwd Packets Length Total  Flow Packets/s  \
0                        1332.0                       0.0        1.444041   
1                         462.0                       0.0        1.096108   
2                         924.0                       0.0        1.963366   
3                        1248.0                       0.0        2.018584   
4                         770.0                       0.0        1.260090   
...                         ...                       ...             ...   
11466                     302.0                       0.0        1.249493   
11467                     348.0                       0.0        1.174449   
11468                     342.0                       0.0      102.875366   
11469                    1092.0                       0.0        1.845081   
11470                     528.0                       0.0        0.863852   

       Flow IAT Mean  Flow IAT Std  Flow IAT Max  Flow IAT Min  ...  \
0       8.310012e+05  7.150903e+05     1756286.0       20328.0  ...   
1       1.368478e+06  1.753121e+06     2608122.0      128834.0  ...   
2       6.111952e+05  8.779418e+05     2167663.0      104515.0  ...   
3       5.661677e+05  5.147874e+05     1624357.0       11845.0  ...   
4       9.919928e+05  1.274903e+06     2860094.0       25727.0  ...   
...              ...           ...           ...           ...  ...   
11466   1.600649e+06  0.000000e+00     1600649.0     1600649.0  ...   
11467   1.702926e+06  0.000000e+00     1702926.0     1702926.0  ...   
11468   1.944100e+04  0.000000e+00       19441.0       19441.0  ...   
11469   6.323120e+05  7.286191e+05     2008603.0       19098.0  ...   
11470   1.736410e+06  1.495010e+06     2793541.0      679278.0  ...   

       Init Fwd Win Bytes  Init Bwd Win Bytes  Active Mean  Active Std  \
0                       0                   0            0           0   
1                       0                   0            0           0   
2                       0                   0            0           0   
3                       0                   0            0           0   
4                       0                   0            0           0   
...                   ...                 ...          ...         ...   
11466                   0                   0            0           0   
11467                   0                   0            0           0   
11468                   0                   0            0           0   
11469                   0                   0            0           0   
11470                   0                   0            0           0   

       Active Max  Active Min  Idle Mean  Idle Std  Idle Max  Idle Min  
0               0           0          0         0         0         0  
1               0           0          0         0         0         0  
2               0           0          0         0         0         0  
3               0           0          0         0         0         0  
4               0           0          0         0         0         0  
...           ...         ...        ...       ...       ...       ...  
11466           0       

In [67]:
pd.DataFrame.from_dict()

AttributeError: module 'pandas' has no attribute 'from_dict'

In [74]:
scaler = StandardScaler().fit(combined_train)

In [75]:
train_features = scaler.transform(combined_train)

In [78]:
combined_test.drop(index=[1771], errors='ignore', inplace=True)
combined_test = combined_test.reset_index(drop=True)

In [79]:
test_features = scaler.transform(combined_test)

In [80]:
train_labels = labels_train.values
test_labels = labels_test.values

In [13]:
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)

NameError: name 'train_features' is not defined

In [34]:
dataset = TensorDataset(train_features_tensor)

In [35]:
val_features_tensor = torch.tensor(test_features, dtype=torch.float32)

In [36]:
loader = DataLoader(dataset, batch_size=512, shuffle=True)

In [79]:
val_dataset = TensorDataset(val_features_tensor)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=True)

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
input_dim = train_features_tensor.shape[1]

NameError: name 'train_features_tensor' is not defined

In [67]:
model = Autoencoder(input_dim=input_dim, latent_dim=16).to(device)

In [68]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [69]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0.0
    for (batch_x,) in loader:
        batch_x = batch_x.to(device)
        optimizer.zero_grad()
        recon, _ = model(batch_x)
        loss = criterion(recon, batch_x)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_x.size(0)

    avg_loss = total_loss / len(loader.dataset)
    print(f'Epoch {epoch}/{num_epochs}, Pre-training Loss: {avg_loss:.6f}')

Epoch 1/10, Pre-training Loss: 0.304340
Epoch 2/10, Pre-training Loss: 0.211980
Epoch 3/10, Pre-training Loss: 0.146398
Epoch 4/10, Pre-training Loss: 0.167516
Epoch 5/10, Pre-training Loss: 0.120737
Epoch 6/10, Pre-training Loss: 0.129820
Epoch 7/10, Pre-training Loss: 0.125069
Epoch 8/10, Pre-training Loss: 0.110301
Epoch 9/10, Pre-training Loss: 0.084055
Epoch 10/10, Pre-training Loss: 0.056656


In [75]:
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=53, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=16, bias=True)
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=53, bias=True)
  )
)

In [84]:
torch.save(model.state_dict(), 'autoencoder_pretrained.pth')
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [81]:
criterion = nn.MSELoss()
val_loss = 0.0
with torch.no_grad():
    for (batch_x,) in val_loader:
        batch_x = batch_x.to(device)
        recon, _ = model(batch_x)
        val_loss += criterion(recon, batch_x).item() * batch_x.size(0)
val_loss /= len(val_loader.dataset)
print(f'Validation MSE Loss: {val_loss:.6f}')

Validation MSE Loss: 0.023940


In [381]:
ae = Autoencoder(input_dim=input_dim, latent_dim=16).to(device)
state = torch.load('autoencoder_pretrained.pth', weights_only=True)
ae.load_state_dict(state)

<All keys matched successfully>

In [382]:
class Attention(nn.Module):
    def __init__(self, hidden_dim: int):
        super().__init__()
        # map each hidden state to a scalar score
        self.score_layer = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, lstm_outputs: torch.Tensor, lengths: torch.Tensor):
        """
        lstm_outputs: (batch, seq_len, hidden_dim)
        lengths:      (batch,) actual lengths before padding
        """
        # 1) compute unnormalized scores for each timestep
        #    shape -> (batch, seq_len, 1) -> squeeze -> (batch, seq_len)
        scores = self.score_layer(lstm_outputs).squeeze(-1)

        # 2) mask out padding positions so they get zero attention
        max_len = lstm_outputs.size(1)
        mask = (
            torch.arange(max_len, device=lengths.device)
            .unsqueeze(0)
            .expand(len(lengths), max_len)
            >= lengths.unsqueeze(1)
        )
        scores = scores.masked_fill(mask, float('-inf'))

        # 3) normalize scores into attention weights
        attn_weights = torch.softmax(scores, dim=1)       # (batch, seq_len)

        # 4) compute weighted sum of LSTM outputs
        #    (batch, 1, seq_len) @ (batch, seq_len, hidden_dim) -> (batch, 1, hidden_dim)
        context = torch.bmm(attn_weights.unsqueeze(1), lstm_outputs).squeeze(1)
        return context, attn_weights

In [383]:
class AE_RNN_Classifier(nn.Module):
    def __init__(self, pretrained_encoder: nn.Module, latent_dim: int,
                 rnn_hidden_dim: int, num_classes: int):
        super().__init__()
        self.encoder = pretrained_encoder
        self.rnn = nn.LSTM(input_size=latent_dim,
                           hidden_size=rnn_hidden_dim,
                           batch_first=True, num_layers=2, dropout=0.4)
        self.attention = Attention(rnn_hidden_dim)
        self.fc = nn.Sequential(
            nn.Dropout(0.45),
            nn.Linear(rnn_hidden_dim, num_classes)
        )

    def forward(self, x, lengths):
        b, t, f = x.size()
        x_flat = x.view(b*t, f)
        z_flat = self.encoder(x_flat)          
        z = z_flat.view(b, t, -1)  
        packed = pack_padded_sequence(z, lengths.cpu(), batch_first=True, enforce_sorted=False)
        out, (h_n, c_n) = self.rnn(packed)  
        lstm_out, _ = pad_packed_sequence(out, batch_first=True)
        context, attn_weights = self.attention(lstm_out, lengths)                     
        return self.fc(context)                 

In [384]:
pretrained_encoder = ae.encoder

In [385]:
for p in pretrained_encoder.parameters():
    p.requires_grad = False

In [386]:
latent_dim = 16

In [387]:
class_counts = [labels_train[labels_train == 0].count(), labels_train[labels_train == 1].count()]

In [388]:
class_counts

[430184, 106339]

In [389]:
total = sum(class_counts)
class_weights = torch.tensor([total/c for c in class_counts], dtype=torch.float32).to(device)

In [390]:
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [391]:
model = AE_RNN_Classifier(
    pretrained_encoder=pretrained_encoder,
    latent_dim=latent_dim,
    rnn_hidden_dim=64,
    num_classes=2
).to(device)

In [392]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                             lr=1e-3,
                             weight_decay=5e-3)

In [393]:
class SequenceDataset(Dataset):
    def __init__(self, sequences_np, labels_np):
        self.seqs = [torch.tensor(s, dtype=torch.float32) for s in sequences_np]
        self.labels = torch.tensor(labels_np, dtype=torch.long)

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        seq = self.seqs[idx]
        return seq, self.labels[idx], seq.size(0)

def collate_fn(batch):
    batch.sort(key=lambda x: x[2], reverse=True)
    seqs, labels, lengths = zip(*batch)
    padded = nn.utils.rnn.pad_sequence(seqs, batch_first=True)
    return padded, torch.tensor(labels), torch.tensor(lengths)

In [394]:
def make_sequences(features: np.ndarray,
                   labels: np.ndarray,
                   seq_len: int = 20,
                   stride: int = 10,
                   label_strategy: str = 'any',
                   threshold: float = 0.0):
    windows = []
    sequence_labels = []
    num_flows = features.shape[0]

    for start in range(0, num_flows - seq_len + 1, stride):
        win = features[start:start + seq_len]
        win_labels = labels[start:start + seq_len]

        if label_strategy == 'any':
            # Label as Attack if any flow in the window is malicious
            seq_label = int(win_labels.any())
        elif label_strategy == 'threshold':
            # Label as Attack if proportion > threshold
            seq_label = int(win_labels.mean() > threshold)
        else:
            raise ValueError(f"Unknown label_strategy: {label_strategy}")

        windows.append(win)
        sequence_labels.append(seq_label)

    sequences = np.stack(windows, axis=0)
    sequence_labels = np.array(sequence_labels, dtype=np.int64)
    return sequences, sequence_labels

In [398]:
seq_train, seq_labels_train = make_sequences(train_features, labels_train.to_numpy(), label_strategy='threshold', threshold=0.2)
seq_test, seq_labels_test = make_sequences(test_features, labels_test.to_numpy(), label_strategy='threshold', threshold=0.2)

In [399]:
len(seq_train)

53651

In [400]:
len(seq_test)

17778

In [401]:
len(seq_labels_train) - sum(seq_labels_train)

45439

In [402]:
sum(seq_labels_train)

8212

In [403]:
classes = np.unique(seq_labels_train)
weights = compute_class_weight("balanced",
                               classes=classes,
                               y=seq_labels_train)
# e.g. weights = [w_benign, w_attack]
weights = [1, 1]
criterion = nn.CrossEntropyLoss(weight=torch.tensor(weights, dtype=torch.float32).to(device))

In [404]:
weights

[1, 1]

In [405]:
seq_labels_test.all() == 1

False

In [406]:
ds_train = SequenceDataset(seq_train, seq_labels_train)
train_loader = DataLoader(ds_train, batch_size=32, shuffle=True, collate_fn=collate_fn)

ds_val = SequenceDataset(seq_test, seq_labels_test)
val_loader = DataLoader(ds_val, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [407]:
num_epochs_stage1 = 5
num_epochs_stage2 = 10

In [408]:
for epoch in range(1, num_epochs_stage1 + 1):
    # Training
    model.train()
    train_loss = 0.0
    for seq_batch, label_batch, lengths in train_loader:
        seq_batch  = seq_batch.to(device)
        label_batch= label_batch.to(device)
        lengths    = lengths.to(device)

        optimizer.zero_grad()
        logits = model(seq_batch, lengths)
        loss   = criterion(logits, label_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
        optimizer.step()
        train_loss += loss.item() * seq_batch.size(0)

    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for seq_batch, label_batch, lengths in val_loader:
            seq_batch  = seq_batch.to(device)
            label_batch= label_batch.to(device)
            lengths    = lengths.to(device)

            logits = model(seq_batch, lengths)
            loss   = criterion(logits, label_batch)
            val_loss += loss.item() * seq_batch.size(0)

    val_loss /= len(val_loader.dataset)

    print(f"[Stage 1] Epoch {epoch}/{num_epochs_stage1} "
          f"Train Loss: {train_loss:.10f}  Val Loss: {val_loss:.10f}")

[Stage 1] Epoch 1/5 Train Loss: 0.0730666736  Val Loss: 0.2654175126
[Stage 1] Epoch 2/5 Train Loss: 0.0543081026  Val Loss: 0.3091011055
[Stage 1] Epoch 3/5 Train Loss: 0.0528095997  Val Loss: 0.2560891643
[Stage 1] Epoch 4/5 Train Loss: 0.0524960023  Val Loss: 0.2831628725
[Stage 1] Epoch 5/5 Train Loss: 0.0519318482  Val Loss: 0.2550412398


In [409]:
for p in model.encoder.parameters():
    p.requires_grad = True
# lower LR if desired
for g in optimizer.param_groups:
    g['lr'] = 1e-5

In [410]:
for epoch in range(1, num_epochs_stage2 + 1):
    # Training
    model.train()
    train_loss = 0.0
    for seq_batch, label_batch, lengths in train_loader:
        seq_batch = seq_batch.to(device)
        label_batch = label_batch.to(device)
        lengths = lengths.to(device)

        optimizer.zero_grad()
        logits = model(seq_batch, lengths)
        loss = criterion(logits, label_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
        optimizer.step()
        train_loss += loss.item() * seq_batch.size(0)

    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for seq_batch, label_batch, lengths in val_loader:
            seq_batch = seq_batch.to(device)
            label_batch = label_batch.to(device)
            lengths = lengths.to(device)

            logits = model(seq_batch, lengths)
            loss = criterion(logits, label_batch)
            val_loss += loss.item() * seq_batch.size(0)

    val_loss /= len(val_loader.dataset)

    print(f"[Stage 2] Epoch {epoch}/{num_epochs_stage2} "
          f"Train Loss: {train_loss:.10f}  Val Loss: {val_loss:.10f}")

[Stage 2] Epoch 1/10 Train Loss: 0.0502368911  Val Loss: 0.2646496811
[Stage 2] Epoch 2/10 Train Loss: 0.0492191202  Val Loss: 0.2660076094
[Stage 2] Epoch 3/10 Train Loss: 0.0494543257  Val Loss: 0.2652734479
[Stage 2] Epoch 4/10 Train Loss: 0.0491675543  Val Loss: 0.2629839968
[Stage 2] Epoch 5/10 Train Loss: 0.0488816158  Val Loss: 0.2638872478
[Stage 2] Epoch 6/10 Train Loss: 0.0489065900  Val Loss: 0.2656942867
[Stage 2] Epoch 7/10 Train Loss: 0.0488399305  Val Loss: 0.2640786367
[Stage 2] Epoch 8/10 Train Loss: 0.0490107297  Val Loss: 0.2621220490
[Stage 2] Epoch 9/10 Train Loss: 0.0489765199  Val Loss: 0.2616804917
[Stage 2] Epoch 10/10 Train Loss: 0.0486052842  Val Loss: 0.2599824179


In [411]:
model.eval()
all_preds = []
all_labels = []

class_names = ["Benign", "Attack"]


with torch.no_grad():
    for seq_batch, label_batch, lengths in val_loader:
        seq_batch = seq_batch.to(device)
        lengths = lengths.to(device)
        logits = model(seq_batch, lengths)
        preds = torch.argmax(logits, dim=1)       
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(label_batch.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

acc = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {acc:.4f}")

# 2) Precision, Recall, F1 per class
prec, rec, f1, support = precision_recall_fscore_support(
    all_labels, all_preds, labels=range(len(class_names))
)
for i, cls in enumerate(class_names):
    print(f"{cls:10}  Precision: {prec[i]:.4f}  Recall: {rec[i]:.4f}  F1: {f1[i]:.4f}  Support: {support[i]}")

# 3) Full classification report
print("\nClassification Report:")
print(classification_report(
    all_labels, all_preds, target_names=class_names, digits=4
))

# 4) Confusion matrix
cm = confusion_matrix(all_labels, all_preds, labels=range(len(class_names)))
print("Confusion Matrix:")
print(cm)

Accuracy: 0.9443
Benign      Precision: 0.9362  Recall: 0.9972  F1: 0.9658  Support: 14018
Attack      Precision: 0.9863  Recall: 0.7468  F1: 0.8500  Support: 3760

Classification Report:
              precision    recall  f1-score   support

      Benign     0.9362    0.9972    0.9658     14018
      Attack     0.9863    0.7468    0.8500      3760

    accuracy                         0.9443     17778
   macro avg     0.9613    0.8720    0.9079     17778
weighted avg     0.9468    0.9443    0.9413     17778

Confusion Matrix:
[[13979    39]
 [  952  2808]]


In [412]:
sum(all_labels == all_preds)/len(all_labels == all_preds)

0.9442569467881652

In [413]:
torch.save(model.state_dict(), 'classifier_rnn_attention.pth')

## Instruction

- Load pretrained autoencoder
- Define the model (AE_RNN_Classifier)
- You will need to import all the implemented classes for the layers and the model
- Load the weights for the model
- Run inference

In [70]:
# Import all the dependensies
import pandas as pd
import numpy as np
import os
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pickle
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight

In [71]:
# Load the first dataset (needs to be in the same folder as ipynb)
hikari = pd.read_csv("ALLFLOWMETER_HIKARI2021.csv")
hikari.drop(columns=["Unnamed: 0.1", "Unnamed: 0"], inplace=True)

In [72]:
# Load the second dataset (needs to be in the same folder as ipynb)
PATH_TO_CIC = "CIC-DDoS2019"
cic_paths = os.listdir(PATH_TO_CIC)
cic_train_paths = [os.path.join(PATH_TO_CIC, path) for path in cic_paths if "-training" in path]
cic_test_paths = [os.path.join(PATH_TO_CIC, path) for path in cic_paths if "-testing" in path]


train_prefixes = [path.split('/')[-1].split('-')[0] for path in cic_train_paths]
test_prefixes = [path.split('/')[-1].split('-')[0] for path in cic_test_paths]

common_prefixes = list(set(train_prefixes).intersection(test_prefixes))

cic_train_paths = [path for path in cic_train_paths if path.split('/')[-1].split('-')[0] in common_prefixes]
cic_test_paths = [path for path in cic_test_paths if path.split('/')[-1].split('-')[0] in common_prefixes]

cic_train = pd.concat([pd.read_parquet(path) for path in cic_train_paths], ignore_index=True)
cic_test = pd.concat([pd.read_parquet(path) for path in cic_test_paths], ignore_index=True)

In [73]:
# Convert key names from dataset 1 to dataset 2 
hikari_to_cic_keys = {
    'flow_duration': 'Flow Duration',
    'fwd_pkts_tot': 'Total Fwd Packets',
    'bwd_pkts_tot': 'Total Backward Packets',
    'fwd_data_pkts_tot': 'Fwd Packets Length Total',
    'bwd_data_pkts_tot': 'Bwd Packets Length Total',
    'fwd_pkts_per_sec': 'Fwd Packets/s',
    'bwd_pkts_per_sec': 'Bwd Packets/s',
    'down_up_ratio': 'Down/Up Ratio',
    'fwd_header_size_tot': 'Fwd Header Length',
    'bwd_header_size_tot': 'Bwd Header Length',
    'flow_pkts_per_sec': 'Flow Packets/s',
    'flow_iat.avg': 'Flow IAT Mean',
    'flow_iat.std': 'Flow IAT Std',
    'flow_iat.max': 'Flow IAT Max',
    'flow_iat.min': 'Flow IAT Min',
    'fwd_iat.tot': 'Fwd IAT Total', 
    'fwd_iat.max': 'Fwd IAT Max',
    'fwd_iat.avg': 'Fwd IAT Mean',
    'fwd_iat.std': 'Fwd IAT Std',
    'fwd_iat.min': 'Fwd IAT Min',
    'bwd_iat.tot': 'Bwd IAT Total',
    'bwd_iat.avg': 'Bwd IAT Mean',
    'bwd_iat.max': 'Bwd IAT Max',
    'bwd_iat.min': 'Bwd IAT Min',
    'bwd_iat.std': 'Bwd IAT Std',
    'flow_SYN_flag_count': 'SYN Flag Count',
    'flow_RST_flag_count': 'RST Flag Count',
    'fwd_PSH_flag_count': 'Fwd PSH Flags',
    'bwd_PSH_flag_count': 'Bwd PSH Flags',
    'flow_ACK_flag_count': 'ACK Flag Count',
    'fwd_URG_flag_count': 'Fwd URG Flags',
    'bwd_URG_flag_count': 'Bwd URG Flags',
    'flow_ECE_flag_count': 'ECE Flag Count',
    'fwd_bulk_rate': 'Fwd Avg Bulk Rate',
    'bwd_bulk_rate': 'Bwd Avg Bulk Rate',
    'fwd_bulk_packets': 'Fwd Avg Packets/Bulk',
    'bwd_bulk_packets': 'Bwd Avg Packets/Bulk',
    'fwd_bulk_bytes': 'Fwd Avg Bytes/Bulk',
    'bwd_bulk_bytes': 'Bwd Avg Bytes/Bulk',
    'active.avg': 'Active Mean',
    'active.min': 'Active Min',
    'active.max': 'Active Max',
    'active.std': 'Active Std',
    'idle.avg': 'Idle Mean',
    'idle.min': 'Idle Min',
    'idle.max': 'Idle Max',
    'idle.std': 'Idle Std',
    'fwd_init_window_size': 'Init Fwd Win Bytes',
    'bwd_init_window_size': 'Init Bwd Win Bytes',
    'fwd_subflow_pkts': 'Subflow Fwd Packets',
    'bwd_subflow_pkts': 'Subflow Bwd Packets',
    'fwd_subflow_bytes': 'Subflow Fwd Bytes',
    'bwd_subflow_bytes': 'Subflow Bwd Bytes',
    'Label': 'Label'
}

In [74]:
# Remove all the keys from dataset 1 that are not overlapping between 2 datasets
hikari_filtered = hikari.loc[:, hikari.columns.intersection(hikari_to_cic_keys.keys())]
# Rename all the overlapping columns in dataset 1 to match names with dataset 2 
hikari_cic = hikari_filtered.rename(columns=hikari_to_cic_keys)

In [75]:
# Remove all the keys from dataset 2 that are not overlapping between 2 datasets
cic_train = cic_train.loc[:, cic_train.columns.intersection(hikari_to_cic_keys.values())]
cic_test = cic_test.loc[:, cic_test.columns.intersection(hikari_to_cic_keys.values())]

In [76]:
# Split the first dataset on train and test (75% - train, 25% - test)
hikari_train = hikari_cic.sample(frac=0.75, random_state=42)
hikari_test = hikari_cic.drop(hikari_train.index)

In [77]:
# Change labels from dataset 2 to be binary (Benign/Attack)
label_conversion_dict = cic_train.Label.unique().tolist() + cic_test.Label.unique().tolist()
label_conversion_dict = list(set(label_conversion_dict))
label_conversion_dict = {
    key: 0 if key == "Benign" else 1 for key in label_conversion_dict
}

cic_train["Label"] = cic_train["Label"].replace(label_conversion_dict)
cic_test["Label"] = cic_test["Label"].replace(label_conversion_dict)

In [78]:
# Combine 2 datasets
combined_train = pd.concat([cic_train, hikari_train])
combined_test = pd.concat([cic_test, hikari_test])

In [79]:
combined_train.columns

Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Fwd Packets Length Total', 'Bwd Packets Length Total',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'SYN Flag Count',
       'RST Flag Count', 'ACK Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
       'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'Init Fwd Win Bytes', 'Init Bwd Win Bytes',
       'Active Mean', 'Active Std', 'Active Max', '

In [98]:
combined_train["Label"] = labels_train

In [99]:
# Separate labels from the features
labels_train = combined_train["Label"]
labels_test = combined_test["Label"]

combined_train.drop(columns=["Label"], inplace=True)
combined_test.drop(columns=["Label"], inplace=True)

In [100]:
# Fit the scaler to normalize the features
scaler = StandardScaler().fit(combined_train)

In [101]:
combined_test.drop(index=[1771], errors='ignore', inplace=True)
combined_test = combined_test.reset_index(drop=True)

In [102]:
# Normalize both train and test features
train_features = scaler.transform(combined_train)
test_features = scaler.transform(combined_test)

In [106]:
# Get size of a feature vector
input_dim = train_features.shape[1]

# Get device for running the models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Latend dimension size of the autoencoder
latent_dim = 16

In [107]:
input_dim

53

In [108]:
# Drop the index from the labels
train_labels = labels_train.values
test_labels = labels_test.values

In [109]:
# Dataset class for sequences
class SequenceDataset(Dataset):
    def __init__(self, sequences_np, labels_np):
        self.seqs = [torch.tensor(s, dtype=torch.float32) for s in sequences_np]
        self.labels = torch.tensor(labels_np, dtype=torch.long)

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        seq = self.seqs[idx]
        return seq, self.labels[idx], seq.size(0)

def collate_fn(batch):
    batch.sort(key=lambda x: x[2], reverse=True)
    seqs, labels, lengths = zip(*batch)
    padded = nn.utils.rnn.pad_sequence(seqs, batch_first=True)
    return padded, torch.tensor(labels), torch.tensor(lengths)

In [110]:
# Logic to make sequences from individual requests
def make_sequences(features: np.ndarray,
                   labels: np.ndarray,
                   seq_len: int = 20,
                   stride: int = 10,
                   label_strategy: str = 'any',
                   threshold: float = 0.0):
    windows = []
    sequence_labels = []
    num_flows = features.shape[0]

    for start in range(0, num_flows - seq_len + 1, stride):
        win = features[start:start + seq_len]
        win_labels = labels[start:start + seq_len]

        if label_strategy == 'any':
            # Label as Attack if any flow in the window is malicious
            seq_label = int(win_labels.any())
        elif label_strategy == 'threshold':
            # Label as Attack if proportion > threshold
            seq_label = int(win_labels.mean() > threshold)
        else:
            raise ValueError(f"Unknown label_strategy: {label_strategy}")

        windows.append(win)
        sequence_labels.append(seq_label)

    sequences = np.stack(windows, axis=0)
    sequence_labels = np.array(sequence_labels, dtype=np.int64)
    return sequences, sequence_labels

In [83]:
# Make sequences out of the datasets
seq_train, seq_labels_train = make_sequences(train_features, labels_train.to_numpy(), label_strategy='threshold', threshold=0.2)
seq_test, seq_labels_test = make_sequences(test_features, labels_test.to_numpy(), label_strategy='threshold', threshold=0.2)

NameError: name 'make_sequences' is not defined

In [84]:
# Make dataloaders for train and test
ds_train = SequenceDataset(seq_train, seq_labels_train)
train_loader = DataLoader(ds_train, batch_size=32, shuffle=True, collate_fn=collate_fn)

ds_val = SequenceDataset(seq_test, seq_labels_test)
val_loader = DataLoader(ds_val, batch_size=1, shuffle=True, collate_fn=collate_fn)

NameError: name 'SequenceDataset' is not defined

In [113]:
# Loading the autoencoder 
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),  
            nn.ReLU(),
            nn.Linear(128, latent_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
            # no activation (assuming using MSELoss for reconstruction)
        )
    def forward(self, x):
        latent = self.encoder(x)
        recon = self.decoder(latent)
        return recon, latent

ae = Autoencoder(input_dim=input_dim, latent_dim=16).to(device)
state = torch.load('autoencoder_pretrained.pth', weights_only=True)
ae.load_state_dict(state)
pretrained_encoder = ae.encoder

In [81]:
# Classes for the model and it's layers 
class Attention(nn.Module):
    def __init__(self, hidden_dim: int):
        super().__init__()
        # map each hidden state to a scalar score
        self.score_layer = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, lstm_outputs: torch.Tensor, lengths: torch.Tensor):
        """
        lstm_outputs: (batch, seq_len, hidden_dim)
        lengths:      (batch,) actual lengths before padding
        """
        # 1) compute unnormalized scores for each timestep
        #    shape -> (batch, seq_len, 1) -> squeeze -> (batch, seq_len)
        scores = self.score_layer(lstm_outputs).squeeze(-1)

        # 2) mask out padding positions so they get zero attention
        max_len = lstm_outputs.size(1)
        mask = (
            torch.arange(max_len, device=lengths.device)
            .unsqueeze(0)
            .expand(len(lengths), max_len)
            >= lengths.unsqueeze(1)
        )
        scores = scores.masked_fill(mask, float('-inf'))

        # 3) normalize scores into attention weights
        attn_weights = torch.softmax(scores, dim=1)       # (batch, seq_len)

        # 4) compute weighted sum of LSTM outputs
        #    (batch, 1, seq_len) @ (batch, seq_len, hidden_dim) -> (batch, 1, hidden_dim)
        context = torch.bmm(attn_weights.unsqueeze(1), lstm_outputs).squeeze(1)
        return context, attn_weights

class AE_RNN_Classifier(nn.Module):
    def __init__(self, pretrained_encoder: nn.Module, latent_dim: int,
                 rnn_hidden_dim: int, num_classes: int):
        super().__init__()
        self.encoder = pretrained_encoder
        self.rnn = nn.LSTM(input_size=latent_dim,
                           hidden_size=rnn_hidden_dim,
                           batch_first=True, num_layers=2, dropout=0.4)
        self.attention = Attention(rnn_hidden_dim)
        self.fc = nn.Sequential(
            nn.Dropout(0.45),
            nn.Linear(rnn_hidden_dim, num_classes)
        )

    def forward(self, x, lengths):
        b, t, f = x.size()
        x_flat = x.view(b*t, f)
        z_flat = self.encoder(x_flat)          
        z = z_flat.view(b, t, -1) 
        packed = pack_padded_sequence(z, lengths.cpu(), batch_first=True, enforce_sorted=False)
        out, (h_n, c_n) = self.rnn(packed)  
        lstm_out, _ = pad_packed_sequence(out, batch_first=True)
        context, attn_weights = self.attention(lstm_out, lengths)                     
        return self.fc(context)  

# Define the model
model = AE_RNN_Classifier(
    pretrained_encoder=pretrained_encoder,
    latent_dim=latent_dim,
    rnn_hidden_dim=64,
    num_classes=2
).to(device)

NameError: name 'pretrained_encoder' is not defined

In [82]:
# Load the weights for the model
state = torch.load('classifier_rnn_attention.pth', weights_only=True)
model.load_state_dict(state)

NameError: name 'model' is not defined

In [119]:
# Run the inference 

# This is model evaluation
model.eval()
all_preds = []
all_labels = []

class_names = ["Benign", "Attack"]

# Inference
with torch.no_grad():
    for seq_batch, label_batch, lengths in val_loader:
        seq_batch = seq_batch.to(device)
        lengths = lengths.to(device)
        # Get logits
        logits = model(seq_batch, lengths)
        # Get predictions
        preds = torch.argmax(logits, dim=1)       
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(label_batch.numpy())
# ==================================================
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

acc = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {acc:.4f}")

# 2) Precision, Recall, F1 per class
prec, rec, f1, support = precision_recall_fscore_support(
    all_labels, all_preds, labels=range(len(class_names))
)
for i, cls in enumerate(class_names):
    print(f"{cls:10}  Precision: {prec[i]:.4f}  Recall: {rec[i]:.4f}  F1: {f1[i]:.4f}  Support: {support[i]}")

# 3) Full classification report
print("\nClassification Report:")
print(classification_report(
    all_labels, all_preds, target_names=class_names, digits=4
))

# 4) Confusion matrix
cm = confusion_matrix(all_labels, all_preds, labels=range(len(class_names)))
print("Confusion Matrix:")
print(cm)

Accuracy: 0.9904
Benign      Precision: 0.0000  Recall: 0.0000  F1: 0.0000  Support: 0
Attack      Precision: 1.0000  Recall: 0.9904  F1: 0.9952  Support: 1145

Classification Report:
              precision    recall  f1-score   support

      Benign     0.0000    0.0000    0.0000         0
      Attack     1.0000    0.9904    0.9952      1145

    accuracy                         0.9904      1145
   macro avg     0.5000    0.4952    0.4976      1145
weighted avg     1.0000    0.9904    0.9952      1145

Confusion Matrix:
[[   0    0]
 [  11 1134]]


/home/razaare/anaconda3/envs/currux/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/razaare/anaconda3/envs/currux/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/razaare/anaconda3/envs/currux/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/